In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
!pip install --quiet cornac

In [ ]:
import cornac
from cornac.data import Reader
from cornac.eval_methods import RatioSplit
from cornac.models import VAECF, BiVAECF

SEED = 42
VERBOSE = True

FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.


In [ ]:
reader = Reader()
train_data = reader.read('/content/gdrive/MyDrive/RecSys Project 2/goodbooks-10k-dataset/ratings.csv', sep=',', skip_lines=1)

In [ ]:
NUM_FACTORS = 50  
AE_LAYERS = [100]  
ACTIVATION = "tanh"  # ["tanh", "sigmoid", "relu", "leaky_relu"]
LIKELIHOOD = "pois"  # ["bern", "mult", "gaus", "pois"]
NUM_EPOCHS = 500  
BATCH_SIZE = 128 
LEARNING_RATE = 5e-4

bivae = BiVAECF(k=NUM_FACTORS, encoder_structure=AE_LAYERS, act_fn=ACTIVATION,
              likelihood=LIKELIHOOD, n_epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,
              learning_rate=LEARNING_RATE, seed=SEED, verbose=VERBOSE, use_gpu=True)

ratio_split = RatioSplit(
  data=train_data, val_size=0.1, test_size=0.1, exclude_unknowns=True, seed=SEED, verbose=VERBOSE
)

prec_20 = cornac.metrics.Precision(k=20)
rec_20 = cornac.metrics.Recall(k=20)
ndcg_20 = cornac.metrics.NDCG(k=20)
ncrr_20 = cornac.metrics.NCRR(k=20)

cornac.Experiment(
  eval_method=ratio_split, models=[bivae], metrics=[prec_20, rec_20, ndcg_20, ncrr_20],
).run()

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 53000
Number of items = 10000
Number of ratings = 783923
Max rating = 5.0
Min rating = 1.0
Global mean = 3.9
---
Test data:
Number of users = 31297
Number of items = 9999
Number of ratings = 97682
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 31241
Number of items = 9999
Number of ratings = 97717
---
Total users = 53000
Total items = 10000

[BiVAECF] Training started!


  0%|          | 0/1000 [00:00<?, ?it/s]


[BiVAECF] Evaluation started!


Ranking:   0%|          | 0/31297 [00:00<?, ?it/s]

Ranking:   0%|          | 0/31241 [00:00<?, ?it/s]


VALIDATION:
...
        | NCRR@20 | NDCG@20 | Precision@20 | Recall@20 | Time (s)
------- + ------- + ------- + ------------ + --------- + --------
BiVAECF |  0.0602 |  0.1015 |       0.0255 |    0.2242 |  54.0151

TEST:
...
        | NCRR@20 | NDCG@20 | Precision@20 | Recall@20 |  Train (s) | Test (s)
------- + ------- + ------- + ------------ + --------- + ---------- + --------
BiVAECF |  0.0598 |  0.1016 |       0.0256 |    0.2262 | 10903.9971 |  57.3576



In [ ]:
bivae.save('\model')

BiVAECF model is saved to \model\BiVAECF\2021-07-14_13-57-21-136973.pkl


'\\model\\BiVAECF\\2021-07-14_13-57-21-136973.pkl'